In [67]:
import pandas as pd

df = pd.read_csv('input_test.csv', encoding='latin1', header=None, names=["text"])
df.head()


,text
0,Long Story short: This game is absolutely wond...
1,Don't be daunted by the price (granted it's su...
2,It's hard to describe the awesomeness of this ...
3,This game is amazing!!!<br /><br />I've been p...
4,Its a solid attempt to turn Dungeons & Dragons...


### Remove punctuation/lower casing

In [68]:
# Load the regular expression library
import re
# Remove punctuation
df['text'] = df['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the to lowercase
df['text'] = df['text'].map(lambda x: x.lower())

### Tokenize

In [3]:
%%time
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = df["text"].values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['long', 'story', 'short', 'this', 'game', 'is', 'absolutely', 'wonderful', 'this', 'according', 'to', 'many', 'board', 'game', 'review', 'sites', 'is', 'set', 'to', 'be', 'the', 'top', 'game', 'of', 'and', 'recommend', 'getting', 'yourself', 'copy', 'though', 'it', 'will', 'likely', 'be', 'difficult', 'take', 'some', 'time', 'its', 'definitely', 'hot', 'selling', 'item', 'and', 'was', 'lucky', 'enough', 'to', 'pre', 'order', 'mine', 'month', 'ago', 'before', 'other', 'folks', 'started', 'seeing', 'it', 'listed', 'on', 'amazon', 'this', 'is', 'definitely', 'worth', 'getting', 'and', 'will', 'make', 'huge', 'hit', 'to', 'your', 'gaming', 'nights', 'more', 'detailed', 'breakdown', 'is', 'below', 'br', 'note', 'relating', 'to', 'pictures', 'the', 'game', 'does', 'not', 'come', 'with', 'painted', 'miniatures', 'as', 'enjoy', 'paining', 'them', 'did', 'so', 'but', 'my', 'wife', 'and', 'played', 'several', 'games', 'before', 'they', 'were', 'painted', 'without', 'any', 'issues', 'confusion'

### Bi-grams and Tri-grams

In [4]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

### Remove stopwords, make bigrams and trigrams, and lemmatization functions definitions

In [5]:
# Uncomment two lines below to download nltk data for first time, after download, comment again

# import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /home/saber/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['long', 'story', 'short', 'game', 'absolutely', 'wonderful', 'accord', 'many', 'board', 'game', 'review', 'site', 'set', 'top', 'game', 'recommend', 'get', 'copy', 'likely', 'difficult', 'take', 'time', 'definitely', 'hot', 'selling', 'item', 'lucky', 'enough', 'pre', 'order', 'mine', 'month', 'ago', 'folk', 'start', 'see', 'list', 'definitely', 'worth', 'get', 'make', 'huge', 'hit', 'gaming', 'night', 'detail', 'breakdown', 'note', 'relate', 'picture', 'game', 'come', 'paint', 'miniature', 'enjoy', 'pain', 'wife', 'play', 'several', 'game', 'paint', 'issue', 'confusion', 'look', 'cool', 'immersive', 'opinion', 'paint', 'mechanic', 'game', 'game', 'reset', 'playthrough', 'rather', 'read', 'story', 'game', 'play', 'build', 'overall', 'hour', 'campaign', 'note', 'campaign', 'legacy', 'game', 'require', 'single', 'playgroup', 'rather', 'start', 'new', 'party', 'anytime', 'drop', 'already', 'exist', 'world', 'plunder', 'dungeon', 'kill', 'baddie', 'game', 'come', 'playable', 'character',

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 2), (3, 1), (4, 2), (5, 4), (6, 2), (7, 4), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 4), (16, 3), (17, 2), (18, 1), (19, 2), (20, 1), (21, 1), (22, 3), (23, 1), (24, 1), (25, 2), (26, 2), (27, 2), (28, 1), (29, 1), (30, 4), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 3), (39, 1), (40, 5), (41, 1), (42, 2), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 3), (57, 2), (58, 17), (59, 1), (60, 1), (61, 1), (62, 1), (63, 22), (64, 2), (65, 2), (66, 1), (67, 1), (68, 3), (69, 3), (70, 2), (71, 1), (72, 1), (73, 8), (74, 1), (75, 5), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 4), (89, 3), (90, 1), (91, 4), (92, 1), (93, 1), (94, 3), (95, 3), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 1), (109, 6), (110, 

### Building the LDA model

In [8]:
# You can change num_topics
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [9]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.126*"game" + 0.067*"play" + 0.055*"family" + 0.052*"love" + 0.032*"fun" + '
  '0.027*"great" + 0.023*"kid" + 0.022*"friend" + 0.017*"age" + 0.015*"buy"'),
 (1,
  '0.067*"player" + 0.062*"game" + 0.021*"card" + 0.014*"strategy" + '
  '0.013*"building" + 0.013*"good" + 0.012*"build" + 0.010*"point" + '
  '0.010*"victory" + 0.010*"turn"'),
 (2,
  '0.058*"game" + 0.041*"piece" + 0.028*"box" + 0.026*"quality" + 0.026*"card" '
  '+ 0.023*"good" + 0.014*"board" + 0.013*"come" + 0.012*"well" + '
  '0.011*"small"'),
 (3,
  '0.038*"card" + 0.028*"player" + 0.018*"point" + 0.016*"turn" + 0.013*"tile" '
  '+ 0.012*"get" + 0.012*"take" + 0.012*"move" + 0.012*"game" + 0.011*"place"'),
 (4,
  '0.126*"game" + 0.047*"play" + 0.032*"easy" + 0.029*"fun" + 0.026*"great" + '
  '0.024*"strategy" + 0.022*"learn" + 0.021*"player" + 0.016*"simple" + '
  '0.015*"lot"'),
 (5,
  '0.105*"game" + 0.084*"play" + 0.031*"time" + 0.026*"get" + 0.026*"fun" + '
  '0.018*"people" + 0.017*"take" + 0.016*"really" 

In [75]:
for index, row in df.iterrows():
    for topic in doc_lda[index]:
        df.at[index, topic[0]] = topic[1]

In [76]:
df.head()

,text,2,3,4,5,6,9,0,1,7,8
0,long story short: this game is absolutely wond...,0.045199,0.191369,0.019837,0.038943,0.457658,0.229318,NaN,NaN,NaN,NaN
1,don't be daunted by the price (granted it's su...,0.027770,0.329543,NaN,0.119259,0.280033,0.238866,NaN,NaN,NaN,NaN
2,it's hard to describe the awesomeness of this ...,0.030453,NaN,NaN,0.227936,0.435922,0.142169,0.045737,0.113474,NaN,NaN
3,this game is amazing<br /><br />i've been play...,0.325481,NaN,0.137774,NaN,0.377719,0.147908,NaN,NaN,NaN,NaN
4,its a solid attempt to turn dungeons & dragons...,NaN,NaN,NaN,NaN,0.383689,0.393845,NaN,0.159137,0.043935,NaN


In [78]:
df.to_csv('results.csv',index=False)